This notebook analyzes data from https://github.com/HuthLab/deep-fMRI-dataset. To set up, see instructions in the `deep-fMRI-dataset` folder.

In [1]:
%load_ext autoreload
%autoreload 2
import datasets
import numpy as np
from os.path import join
# from ridge_utils.SemanticModel import SemanticModel
from matplotlib import pyplot as plt
from typing import List
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer
# from feature_spaces import em_data_dir, data_dir, results_dir
from collections import defaultdict
import pandas as pd
import os
import seaborn as sns
# from fit_linear_models import get_dsets
# from ridge_utils.SemanticModel import SemanticModel
# import encoding_utils, feature_spaces
from transformers.pipelines.pt_utils import KeyDataset
# from feature_spaces import *
from transformers import pipeline
import datasets
import sys
sys.path.append('..')
import data
from tqdm import tqdm

In [28]:
r = defaultdict(list)
for dset in tqdm([
    # 'trec',
    'emotion', 'rotten_tomatoes', 'tweet_eval', 'sst2',
    # 'go_emotions', 'poem_sentiment', 
    # 'ethics-commonsense', 'ethics-deontology', 'ethics-justice', 'ethics-utilitarianism', 'ethics-virtue',
    'probing-subj_number', 'probing-word_content', 'probing-obj_number',
    'probing-past_present', 'probing-sentence_length', 'probing-top_constituents',
    'probing-tree_depth', 'probing-coordination_inversion', 'probing-odd_man_out',
    'probing-bigram_shift', 'moral_stories',
]):
    print(dset)
    X, y, X_test, y_test = data.get_dsets(dataset=dset, seed=1)
    values, counts = np.unique(y, return_counts=True)
    y_majority_frac_train = counts.max() / len(y)
    n_classes = values.size
    values, counts = np.unique(y_test, return_counts=True)
    y_majority_frac_test = counts.max() / len(y)
    lens_train = [len(x) for x in X]

    r['dset'].append(dset)
    r['n_train'].append(len(X))
    r['n_test'].append(len(X_test))
    r['y_majority_frac_train'].append(y_majority_frac_train)
    r['y_majority_frac_test'].append(y_majority_frac_test)
    r['len_avg_train'].append(np.mean(lens_train))
    r['len_std_train'].append(np.std(lens_train))
    r['n_classes'].append(n_classes)
    r['ex1'].append(X[0])


  0%|          | 0/15 [00:00<?, ?it/s]

emotion


Using custom data configuration default
Reusing dataset emotion (/home/chansingh/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)


  0%|          | 0/3 [00:00<?, ?it/s]

Using custom data configuration default
Reusing dataset emotion (/home/chansingh/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)


...

  0%|          | 0/3 [00:00<?, ?it/s]

 20%|██        | 3/15 [00:03<00:13,  1.10s/it]

sst2


Using custom data configuration default
Reusing dataset sst2 (/home/chansingh/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame.from_dict(r)
df.to_pickle('../processed/data_table.pkl')

In [ ]:
df = pd.read_pickle('../processed/data_table.pkl')
df.round(2)

,dset,n_train,n_test,y_majority_frac_train,y_majority_frac_test,len_avg_train,len_std_train,n_classes,ex1
0,emotion,16000,2000,0.34,0.34,96.85,55.90,6,i didnt feel humiliated
1,rotten_tomatoes,8530,1066,0.50,0.50,113.97,51.05,2,the rock is destined to be the 21st century's ...
2,tweet_eval,9000,1000,0.58,0.58,122.84,67.20,2,@user nice new signage. Are you not concerned ...
3,sst2,67349,872,0.56,0.56,53.51,43.41,2,hide new secretions from the parental units
4,probing-subj_number,82010,8088,0.50,0.50,84.63,114.20,2,Coming from a xenophobic race that possesses t...
5,probing-word_content,83868,8357,0.00,0.00,83.86,95.66,1000,"It just hadn 't seemed important, and he didn ..."
6,probing-obj_number,80124,8014,0.50,0.50,83.40,101.86,2,"Money would replace the drugs in the bags, onc..."
7,probing-past_present,85700,8552,0.50,0.50,71.20,75.14,2,"She shone her light around the space, followin..."
8,probing-sentence_length,87408,8653,0.17,0.17,87.04,165.93,6,But it was not here.
9,probing-top_constituents,70185,7451,0.05,0.05,88.59,508.29,20,"I wanted to start asking questions now, but fo..."


# Look at narrative stories

In [5]:
import encoding_utils, feature_spaces
train_stories, test_stories, allstories = encoding_utils.get_allstories([1, 2, 3, 4, 5])

In [6]:
wordseqs = feature_spaces.get_story_wordseqs(allstories)

In [12]:
' '.join(wordseqs['sloth'].data)

"so i'm sitting in my shrink's office this one time and he says to me how's it going and i say well to be honest doc i had a really really horrible experience the other day and he says tell me about it and i say well it all started when i was i was laying on my couch and he looks at me and he says really which if you know me is an incredibly funny thing for him to say because i i have probably spent more time lying on couches than any able bodied man under the age of say a hundred and ten living today uh if lying on couches was a martial art i would have a ninth level black belt ninja master status in it um i uh as a matter of fact when i i was originally asked by the moth to do this event and they explained about the seven deadly sins i was amazed that sloth was even still available uh i i i assumed that sloth would be the first sin picked because it is really the central thing about our culture isn't it it's which is all we live we live in a whole society based on consumer entertainm